In [1]:
# Install and importing neccessary libraries
try:
    import polars as pl
except ImportError:
    !pip install polars
    import polars as pl

import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 19.9 MB/s eta 0:00:00a 0:00:01


In [2]:
# Loading data
train_p1 = pl.read_csv("./Data/Train/Part_1_2years_train_schedule.csv")
train_p2 = pl.read_csv("./Data/Train/Part_2_2years_train_schedule.csv")
train_p3 = pl.read_csv("./Data/Train/Part_3_2years_train_schedule.csv")
train_p4 = pl.read_csv("./Data/Train/Part_4_2years_train_schedule.csv")
train_p5 = pl.read_csv("./Data/Train/Part_5_2years_train_schedule.csv")
df_train = pl.concat([train_p4, train_p3, train_p2, train_p1, train_p5], how = "vertical")
df_train

Station,Date,Arrival,Train Nr.,Departure from,Arrival train station
str,i64,str,str,str,str
"""Absdorf_Hippersdorf""",20230912,"""15:47""","""IC 2173""","""Sigmundsherberg (ab 15:09)""","""Absdorf-Hippersdorf"""
"""Absdorf_Hippersdorf""",20230913,"""15:47""","""IC 2173""","""Sigmundsherberg (ab 15:09)""","""Absdorf-Hippersdorf"""
"""Absdorf_Hippersdorf""",20230914,"""15:47""","""IC 2173""","""Sigmundsherberg (ab 15:09)""","""Absdorf-Hippersdorf"""
"""Absdorf_Hippersdorf""",20230919,"""15:47""","""IC 2173""","""Sigmundsherberg (ab 15:09)""","""Absdorf-Hippersdorf"""
"""Absdorf_Hippersdorf""",20230920,"""15:47""","""IC 2173""","""Sigmundsherberg (ab 15:09)""","""Absdorf-Hippersdorf"""
…,…,…,…,…,…
"""Velden_am_Wörther_See""",20250207,"""22:22""","""IC 784""","""Mallnitz-Obervellach (ab 21:17…","""Velden am Wörther See"""
"""Velden_am_Wörther_See""",20250207,"""22:34""","""RJ 633""","""Wien Hbf (ab 18:24)""","""Velden am Wörther See"""
"""Velden_am_Wörther_See""",20250207,"""23:22""","""RJ 884""","""Spittal-Millstättersee (ab 22:…","""Velden am Wörther See"""


### Whitelist all Austrian Stations 

In [3]:
# Create a Whitelist of all Austrian stations

austrian_stations = [
    "Traun OÖ",
    "Amstetten NÖ",
    "Schwarzach im Pongau-St.Veit",
    "Friesach in Kärnten",
    "St.Michael in Obersteiermark",
    "LinzHbf",
    "Lienz in Osttirol",
    "Baden b.Wien",
    "Micheldorf in OÖ",
    "Bregenz Riedenburg",
    "Vöcklabruck Bahnhof",
    "Feldbach/Raab",
    "Graz Hbf (Busbahnhof)",
    "Bad Mitterndorf Heilbrunn",
    "Hopfgarten im Brixental",
    "Golling/Salzach-Abtenau",
    "Großweikersdorf",
    "Lend/Salzach",
    "Rosenbach bei Villach",
    "Haus im Ennstal",
    "Graz-Liebenau Murpark",
    "Graz Liebenau Murpark",
    "Wien Mitte-Landstraße",
    "Wien Hbf (Autoreisezug)",
    "Rohrendorf b.Krems",
    "Puch b.Hallein Urstein",
    "Wien Hbf (Autoreisezuganlage)",
    "Wien Hbf (Bahnsteige 1-2)",
    "St.Egyden am Steinfeld",
    "Graz Don Bosco (Bahnsteige 1-2)",
    "Steeg/Hallstätter See-Gosau",
    "Ebensee/Traunsee",
    "Klaus an der Pyhrnbahn",
    "Wien Matzleinsdorfer Platz (ÖBB)",
    "Himberg b.Wien",
    "Rohr im Kremstal-Bad Hall",
    "Limberg NÖ-Maissau",
    "Linz(Donau)Hbf",
    "Krems an der Donau",
    "Schlierbach/Krems",
    "Graz Liebenau Murpark (Bahnhst)",
    "Fels am Wagram",
    "Kirchstetten b.Neulengbach",
    "Studenzen-Fladnitz im Raabtal",
    "Tauplitz Schiflugschanze Kulm",
    "Pernegg/Mur",
    "Purkersdorf b.Wien Sanatorium",
    "Gmunden Engelhof",
    "Purkersdorf b.Wien Zentrum",
    "Bruck/Glocknerstraße-Fusch",
    "Wolkersdorf im Weinviertel",
    "Mixnitz Bärenschützklamm",
    "Untertullnerbach",
    "Götzendorf/Leitha",
    "Lahrndorf b.Garsten",
    "Linz-Pichling"
]

stationen = pd.read_csv("./Data/Verzeichnis der Verkehrsstationen.csv", sep = ";")
stationen = stationen.drop(range(1038,1058)) #these are NaNs
replace_map = {
    " a.d.L.": "/Leitha",
    " a.d.": "/", 
    " a d": "/",
    "N.Ö.": "NÖ",
    "Westbf": "Westbahnhof",
    "Ostbf": "Ostbahnhof",
    "Obertrattnach-M Hofk": "Obertrattnach-Markt Hofkirchen",
    "Wien Hauptbahnhof, Wien Hauptbahnhof Südtiroler Platz": "Wien Hbf",
    "Wien Mitte, Wien Mitte - CAT": "Wien Mitte",
    "Wörthersee": "Wörther See",
    "Krumpendorf": "Krumpendorf/Wörthersee"
}

def replace_text(text):
    for old, new in replace_map.items():
        text = text.replace(old, new)
    return text
    
stationen["Verkehrsstation"] = stationen["Verkehrsstation"].apply(replace_text)
stations = sorted(stationen["Verkehrsstation"].tolist())

# For St. variants
st_variants = []
for station in stations:
    if "St." in station: 
        st_variants.append(station.replace("St. ", "St."))
        st_variants.append(station.replace("St.", "St. "))

stations_combined = sorted(list(set(stations + st_variants + austrian_stations)))

df_train_white = df_train.filter(pl.col("Arrival train station").is_in(stations_combined))

### Blacklist Non-Austrian Stations

In [4]:
# Create a blacklist of all Non-Austrian stations found

non_austrian_stations = [
    "Traunstein",
    "Thalwil",
    "Baden(CH)",
    "Baden-Baden",
    "Baden Baden",
    "Plattling",
    "Friedberg(Hess)",
    "Fürth Hbf",
    "Hørning St",
    "Hornberg(Schwarzw)",
    "Hausach",
    "Fürth(Bay)Hbf",
    "Rumia",
    "Felsberg-Wolfershausen",
    "Melkouwen",
    "Felsberg-Gensungen",
    "Hausen (b Düren)",
    "Furth im Wald",
    "Dornach(CH),",
    "Thalheim-Altikon",
    "Schlierbach(Schwalm-Eder-Kr.)",
    "Felsberg-Altenbrunslar",
    "Pittenhart",
    "Gräfendorf",
    "Langenwang(Schwab)"
]
df_train_black = df_train.filter(~pl.col("Arrival train station").is_in(non_austrian_stations))

### Testing Filter for Austrian Stations 

In [9]:
stations_in_train_white = set(df_train_white["Arrival train station"])
stations_in_train_black = set(df_train_black["Arrival train station"])

extra_stations = len(stations_in_train_white - stations_in_train_black)
print("Overlapping Stations:", extra_stations)

Overlapping Stations: 0


# Data preproccessing

In [6]:
df_train = df_train_white
df_train = df_train.rename({"Arrival train station": "Arrival Station", "Station": "Searched Station"}).with_columns(
    [
        # Date: Change Type
        pl.col("Date").cast(pl.Utf8).str.strptime(pl.Date, "%Y%m%d"),
        
        # Scheduled Arrival: Extract and Change Type
        (pl.col("Date").cast(pl.Utf8) + " " + pl.col("Arrival").str.extract(r"^(\d{2}:\d{2})", 1))
        .str.strptime(pl.Datetime, "%Y%m%d %H:%M")
        .alias("Scheduled Arrival Time"),
        
        # Actual Arrival Time: Extract and Change Type
        pl.when(pl.col("Arrival").str.contains("Ausfall"))
        .then(None)
        .when(pl.col("Arrival").str.contains(r"\((\+\d+)\)"))
        .then(
            # Add Delay -> Change of Type needed: Datetime ------------- ChatGPT
            (pl.col("Date").cast(pl.Utf8) + " " + pl.col("Arrival").str.extract(r"^(\d{2}:\d{2})", 1))
            .str.strptime(pl.Datetime, "%Y%m%d %H:%M")
            + pl.duration(minutes=pl.col("Arrival").str.extract(r"\((\+\d+)\)", 1).cast(pl.Int64))
        )
        .otherwise(
            # On-Time
            (pl.col("Date").cast(pl.Utf8) + " " + pl.col("Arrival").str.extract(r"^(\d{2}:\d{2})", 1))
            .str.strptime(pl.Datetime, "%Y%m%d %H:%M")
        )
        .alias("Actual Arrival Time"),
        
        # Departing Station: Extract
        pl.col("Departure from")
        .str.extract(r"^(.*?) \(ab \d{2}:\d{2}\)", 1)
        .alias("Departing Station"),
        
        # Scheduled Departure Time: Extract and Change Type
        pl.col("Departure from")
        .str.extract(r"\(ab (\d{2}:\d{2})\)", 1)
        .str.strptime(pl.Time, "%H:%M")
        .alias("Scheduled Departure Time"),
    ]
)

# Working Variables
df_train = df_train.with_columns(

    # Include Date at Scheduled Departure
    pl.when(pl.col("Scheduled Departure Time") > pl.col("Scheduled Arrival Time").dt.time())
    .then(
        (pl.col("Date").cast(pl.Datetime) + pl.col("Scheduled Departure Time").cast(pl.Duration) - pl.duration(days=1))
    )
    .otherwise(
        (pl.col("Date").cast(pl.Datetime) + pl.col("Scheduled Departure Time").cast(pl.Duration))
    )
    .alias("Scheduled Departure Time"),
    
    # Delay: 
    pl.when(pl.col("Actual Arrival Time").is_not_null())
    .then(pl.col("Actual Arrival Time") - pl.col("Scheduled Arrival Time"))
    .alias("Delay")
    .dt.total_minutes(),

    # Cancellation: 
    pl.when(pl.col("Actual Arrival Time").is_null())
    .then(1)
    .otherwise(0)
    .alias("Cancellation"),

    # On Time
    pl.when((pl.col("Actual Arrival Time").is_not_null()) & 
            (
                (pl.col("Actual Arrival Time") - pl.col("Scheduled Arrival Time"))
                <= pl.duration(minutes = 5) # Thats how ÖBB defines its punctuality: https://infrastruktur.oebb.at/dam/jcr:03a3177a-b9a6-41aa-91cb-43d8e289b11f/zahlen-daten-fakten-folder-2024-oebb.pdf
            )
           )
    .then(1)
    .otherwise(0)
    .alias("On Time"),

    pl.col("Train Nr.").str.split(" ").list.first().alias("Train Type"),

    # Time Variables:
    pl.col("Scheduled Departure Time").dt.hour().alias("Departure Hour"),
    pl.col("Scheduled Arrival Time").dt.hour().alias("Arrival Hour"),
    pl.col("Date").dt.weekday().alias("Day of Week"),
    pl.col("Date").dt.month().alias("Month"),
    pl.col("Date").dt.year().alias("Year")
)

# Delete unnecessary columns
df_train = df_train.drop(["Departure from", "Arrival"])

# Delete doublicated rows
df_train = df_train.unique(
    subset = [
        "Date",
    	"Train Nr.",
        "Train Type",
        "Arrival Station",
        "Scheduled Arrival Time",
        "Actual Arrival Time",
        "Departing Station",
        "Scheduled Departure Time",
        "Delay",
        "Cancellation",
        "On Time",
        "Departure Hour",
        "Arrival Hour",
        "Day of Week",
        "Month",
        "Year"
    ]
)

In [7]:
df_train = df_train.select(
    pl.col(
        [
            "Searched Station",
            "Date", 
            "Train Nr.", 
            "Train Type",
            "Departing Station", 
            "Scheduled Departure Time", 
            "Arrival Station",
            "Scheduled Arrival Time", 
            "Actual Arrival Time", 
            "Delay", 
            "Cancellation", 
            "On Time",
            "Departure Hour",
            "Arrival Hour",
            "Day of Week", 
            "Month",
            "Year"
        ]
    )
)
df_train

Searched Station,Date,Train Nr.,Train Type,Departing Station,Scheduled Departure Time,Arrival Station,Scheduled Arrival Time,Actual Arrival Time,Delay,Cancellation,On Time,Departure Hour,Arrival Hour,Day of Week,Month,Year
str,date,str,str,str,datetime[μs],str,datetime[μs],datetime[μs],i64,i32,i32,i8,i8,i8,i8,i32
"""Landeck_Zams""",2024-09-30,"""RJX 869""","""RJX""","""Bregenz""",2024-09-30 12:40:00,"""Landeck-Zams""",2024-09-30 14:25:00,2024-09-30 14:27:00,2,0,1,12,14,1,9,2024
"""Wien_Hbf""",2025-01-08,"""RJ 130""","""RJ""","""Venezia Santa Lucia""",2025-01-08 15:52:00,"""Wien Hbf""",2025-01-08 23:36:00,2025-01-09 00:10:00,34,0,0,15,23,3,1,2025
"""Attnang_Puchheim""",2024-07-05,"""WB 917""","""WB""","""Salzburg Hbf""",2024-07-05 12:52:00,"""Attnang-Puchheim""",2024-07-05 13:35:00,2024-07-05 13:35:00,0,0,1,12,13,5,7,2024
"""Tullnerfeld""",2024-01-20,"""NJ 446""","""NJ""","""Wien Hbf (Autoreisezuganlage)""",2024-01-20 22:44:00,"""Tullnerfeld""",2024-01-20 23:19:00,2024-01-20 23:20:00,1,0,1,22,23,6,1,2024
"""Schärding""",2024-02-28,"""IC 296""","""IC""","""Fürth Hbf""",2024-02-28 00:33:00,"""Schärding""",2024-02-28 21:25:00,2024-02-28 21:25:00,0,0,1,0,21,3,2,2024
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Wien_Meidling""",2023-06-26,"""NJ 490""","""NJ""","""Wien Hbf""",2023-06-26 19:35:00,"""Wien Meidling""",2023-06-26 19:40:00,2023-06-26 19:41:00,1,0,1,19,19,1,6,2023
"""Wels_Hbf""",2023-05-25,"""RJX 42""","""RJX""","""Mosonmagyarovar""",2023-05-25 19:20:00,"""Wels Hbf""",2023-05-25 22:00:00,2023-05-25 22:00:00,0,0,1,19,22,4,5,2023
"""St._Pölten_Hbf""",2024-12-18,"""EN 406""","""EN""","""Zebrzydowice""",2024-12-18 04:26:00,"""St.Pölten Hbf""",2024-12-18 22:41:00,2024-12-18 23:20:00,39,0,0,4,22,3,12,2024


In [8]:
df_train.write_parquet("./Data/Train/Train_2years.parquet")